In [ ]:
# !pip uninstall opencv-python
# %cd /content/drive/MyDrive/cv2_cuda/
# !cp cv2.cpython-38-x86_64-linux-gnu.so /lib/python3/dist-packages

In [ ]:
import glob
import numpy as np
import tensorflow as tf
# import cv2
# import json
from sklearn.model_selection import train_test_split

# cv2.__version__

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x = np.load('/content/drive/MyDrive/skku_ai/x.npy')
y = np.load('/content/drive/MyDrive/skku_ai/y_6.npy')

Val 1 미세각질
Val 2 피지과다
Val 3 모낭사이홍반
Val 4 모낭홍반/농포
Val 5 비듬
Val 6 탈모

각각 4가지 상태가 있는데, 이를 모두 고려한다면 4096 가지의 classification을 진행해야함. 따라서 각 상태에 대해서 증상 유, 무만 고려할 예정.

In [ ]:
len(x)

18513

In [ ]:
x.shape

(18513, 224, 224, 3)

In [ ]:
y.shape

(18513, 1)

In [ ]:
x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, stratify=y,random_state=42)
# y_train, y_test = train_test_split(y, test_size=0.2,,random_state=42)


# 전체 데이터에서 대부분이 탈모라고 진단하기 때문에 문제가 발생했다

1. 97%라는 수치로 고정된 것은 label == 1 이 전체 데이터에서 92% 존재했기 때문이다

2. 따라서 데이터 증강을 통해 해결해야한다.

3. 모든 데이터에 증강을 해도 결국 비율상 같은 문제가 발생할 수 있기 때문에 적은 라벨을 가진 data를 증강해야한다.

4. Rotate, filter, flip을 통해 증강을 진행할건데 어떻게 변할지 예상이 안된다.

5. 증강이 결국 같은 특징을 공유하는 이미지를 추가하는 것 인데 결과가 어떨지 잘 모르겠다.

In [ ]:
np.count_nonzero(y_test), y_test.shape, np.count_nonzero(y_train), y_train.shape

(3596, (3703, 1), 14383, (14810, 1))

In [ ]:

model1 = tf.keras.models.Sequential()
#
# model1.add(tf.keras.layers.RandomBrightness(0.2,training=True)
model1.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',input_shape=(224,224,3)))
# model1.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu'))  # conv2d 수행하면 64채널의 결과가 나오고, kernelsize = 3
  # conv2d 수행하면 64채널의 결과가 나오고, kernelsize = 3
# model1.add(tf.keras.layers.Dropout(rate=0.5))                          #  DO1
model1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#
model1.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model1.add(tf.keras.layers.Dropout(rate=0.5))                          # DO2
model1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#
model1.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Dropout(rate=0.5))                           # DO3
model1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#

model1.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
# model1.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
model1.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(100, activation='relu'))
model1.add(tf.keras.layers.Dense(4, activation='softmax'))
#
model1.summary()

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 dropout (Dropout)           (None, 112, 112, 128)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 256)       2

In [ ]:
saved_model = '/content/drive/MyDrive/skku_ai/test1.h5'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10,restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint(saved_model, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
               
#                loss='sparse_categorical_crossentropy',
#                metrics=['accuracy'])

history = model1.fit(x_train, y_train,
        epochs = 25,
        callbacks=[es, mc],
        batch_size = 32,
        validation_data = (x_test, y_test))

Epoch 1/25
463/463 [==============================] - ETA: 0s - loss: 10.0997 - accuracy: 0.7103
Epoch 1: val_accuracy improved from -inf to 0.72104, saving model to /content/drive/MyDrive/skku_ai/test1.h5
463/463 [==============================] - 79s 147ms/step - loss: 10.0997 - accuracy: 0.7103 - val_loss: 0.9386 - val_accuracy: 0.7210
Epoch 2/25
 24/463 [>.............................] - ETA: 55s - loss: 0.7691 - accuracy: 0.7448

KeyboardInterrupt: ignored

In [ ]:
x_v = np.load('/content/drive/MyDrive/skku_ai/x_v.npy')
y_v = np.load('/content/drive/M_vDrive/skku_ai/y_v.npy')